## Setup

In [ ]:
import pandas as pd
import sqlite3 as s3
import pyodbc
import warnings
warnings.simplefilter('ignore')

DB = {'servername': 'NOAH\SQLEXPRESS01',
      'database': '4.3 db'}

export_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER=' + DB['servername'] + 
                            ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
export_cursor = export_conn.cursor()
export_cursor

go_sales = s3.connect('go_sales.sqlite')
go_staff = s3.connect('go_staff.sqlite')
go_crm = s3.connect('go_crm.sqlite')

product = pd.read_sql_query('SELECT * FROM product', go_sales)
product_type = pd.read_sql_query('SELECT * FROM product_type', go_sales)
sales_staff = pd.read_sql_query('SELECT * FROM sales_staff', go_sales)
sales_branch = pd.read_sql_query('SELECT * FROM sales_branch', go_sales)
staff_manager = pd.read_sql_query('SELECT * FROM sales_staff', go_staff)
order_method = pd.read_sql_query('SELECT * FROM order_method', go_sales)
return_reason = pd.read_sql_query('SELECT * FROM return_reason', go_sales)
retailer_contact = pd.read_sql_query('SELECT * FROM retailer_contact', go_crm)

### PRODUCT

In [ ]:
merged = pd.merge(product, product_type, left_on='PRODUCT_TYPE_CODE', how='inner', right_on='PRODUCT_TYPE_CODE')
new_product = pd.DataFrame(merged[['PRODUCT_NUMBER', 'PRODUCT_NAME', 'DESCRIPTION', 'INTRODUCTION_DATE', 'PRODUCT_TYPE_CODE', 'PRODUCTION_COST', 'MARGIN', 'PRODUCT_IMAGE', 'LANGUAGE']])
new_product

for index, row in new_product.iterrows():
    try:
        query = f"INSERT INTO product VALUES ({row['PRODUCT_NUMBER']}, '{row['PRODUCT_NAME']}', '{row['DESCRIPTION']}', '{row['INTRODUCTION_DATE']}', '{row['PRODUCT_TYPE_CODE']}', '{row['PRODUCTION_COST']}', '{row['MARGIN']}', '{row['PRODUCT_IMAGE']}', '{row['LANGUAGE']}')"
        export_cursor.execute(query)
    except pyodbc.Error:
        print(query)

export_conn.commit()
export_cursor.close()

### SALES_STAFF

In [ ]:
merged = pd.merge(staff_manager, sales_branch, left_on='SALES_BRANCH_CODE', how='inner', right_on='SALES_BRANCH_CODE')
new_staff = pd.DataFrame(merged[['SALES_STAFF_CODE', 'WORK_PHONE', 'FAX', 'EMAIL', 'FIRST_NAME', 'LAST_NAME', 'POSITION_EN', 'EXTENSION', 'DATE_HIRED', 'SALES_BRANCH_CODE', 'MANAGER_CODE']])
new_staff

for index, row in new_staff.iterrows():
    try:
        query = f"INSERT INTO sales_staff VALUES ({row['SALES_STAFF_CODE']}, '{row['WORK_PHONE']}', '{row['FAX']}', '{row['EMAIL']}', '{row['FIRST_NAME']}', '{row['LAST_NAME']}', '{row['POSITION_EN']}', '{row['EXTENSION']}', '{row['DATE_HIRED']}', '{row['SALES_BRANCH_CODE']}', '{row['MANAGER_CODE']}')"
        export_cursor.execute(query)
    except pyodbc.Error:
        print(query)

export_conn.commit()
export_cursor.close()

### ORDER_METHOD

In [ ]:
new_order_method = order_method[['ORDER_METHOD_CODE', 'ORDER_METHOD_EN']]
new_order_method

for index, row in new_order_method.iterrows():
    try:
        query = f"INSERT INTO order_method VALUES ({row['ORDER_METHOD_CODE']}, '{row['ORDER_METHOD_EN']}')"
        export_cursor.execute(query)
    except pyodbc.Error:
        print(query)

export_conn.commit()
export_cursor.close()

### RETURN_REASON

In [ ]:
new_return_reason = return_reason[['RETURN_REASON_CODE', 'RETURN_DESCRIPTION_EN']]
new_return_reason

for index, row in new_return_reason.iterrows():
    try:
        query = f"INSERT INTO return_reason VALUES ({row['RETURN_REASON_CODE']}, '{row['RETURN_DESCRIPTION_EN']}')"
        export_cursor.execute(query)
    except pyodbc.Error:
        print(query)

export_conn.commit()
export_cursor.close()

### RETAILER_CONTACT

In [ ]:
new_retailer_contact = retailer_contact[['RETAILER_CONTACT_CODE', 'FAX', 'E_MAIL', 'RETAILER_SITE_CODE', 'FIRST_NAME', 'LAST_NAME', 'JOB_POSITION_EN', 'EXTENSION', 'GENDER']]
new_retailer_contact

for index, row in new_retailer_contact.iterrows():
    try:
        last_name = row['LAST_NAME'].replace("'", "''")
        query = f"INSERT INTO return_reason VALUES ({row['RETAILER_CONTACT_CODE']}, '{row['FAX']}', '{row['EMAIL']}', '{row['RETAILER_SITE_CODE']}', '{row['FIRST_NAME']}', '{last_name}', '{row['JOB_POSITION_EN']}', '{row['EXTENSION']}', '{row['GENDER']}')"
        export_cursor.execute(query)
    except pyodbc.Error:
        print(query)

export_conn.commit()
export_cursor.close()